In [1]:
import numpy as np
import numpy.typing as npt
# from numba import jit, njit, prange
import pickle
from dataclasses import dataclass, asdict, field
import argparse
import hashlib
import json
from os import listdir
from os.path import isfile, join
# import pandas as pd
from pathlib import Path
from typing import Any
import time
import sys
import copy
import concurrent.futures as cf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable

from src.dataclass import (
    Input, Lattice, Parameter, Train, Save,
    Processed_Input, Topology, Conjugate, Result
)
from src.manage_data import save_result, save_log, load_result
from src.initial_state import get_initial_state
from src.process_input import (get_processed_input, get_T_and_H, get_J)
from src.metropolis import execute_metropolis_update
from src.function import (time_correlation, column_average_2d, get_spin_glass)
from src.process_output import get_result, get_animation

In [2]:
def run_ensemble(
    input: Input, 
    processed_input: Processed_Input,
    ensemble_num: int
) -> tuple[int, tuple, npt.NDArray[np.float64], int]:
    
    (size, dimension, iteration, sweep, 
     measurement, interval, recent, threshold) = (
        input.lattice.size,
        input.lattice.dimension,
        input.train.iteration,
        input.train.sweep,
        input.train.measurement,
        input.train.interval,
        input.train.recent,
        input.train.threshold,
    )
    
    J = get_J(input, processed_input)
    
    begin = time.perf_counter()
    initial = get_initial_state(input)
    update = initial.copy()
    autocorr, autocorr_len = np.empty(iteration+1, dtype=np.float64), 1
    autocorr[0] = time_correlation(initial, initial, size**dimension)
    raw_output = np.empty((measurement, size**dimension), dtype=np.complex128)

    now = time.perf_counter()
    # Removing initial state effect until autoautocorrelation satsisfies certain criteria
    for _ in range(iteration):
        update = execute_metropolis_update(
            input, processed_input, J, update)
        autocorr[autocorr_len] = np.abs(time_correlation(
            update, initial, size**dimension))
        autocorr_len += 1
        if autocorr_len > recent:
            temp = autocorr[autocorr_len-recent:autocorr_len]
            if np.average(temp) < threshold or np.std(temp) < threshold:
                break
    
    temp = autocorr[autocorr_len-recent:autocorr_len]
    if(ensemble_num == 1):
        print(
            f"ensemble {ensemble_num} initial effect removed, "
            f"time: {time.perf_counter()-now}s, "
            f"iter: {autocorr_len-1}, corr: {np.average(temp):.4f}, std: {np.std(temp):.4f}"
        )

    now = time.perf_counter()
    # Collect raw output after performing metropolis update
    for i in range(sweep):
        update = execute_metropolis_update(input, processed_input, J, update)
        if autocorr_len <= iteration:
            autocorr[autocorr_len] = np.abs(time_correlation(update, initial, size**dimension))
            autocorr_len += 1
        if i % interval == interval - 1:
            raw_output[int(i/interval)] = update.copy()

    if(ensemble_num == 1):
        print(
            f"ensemble {ensemble_num} metropolis updated, time:"
            f" {time.perf_counter()-now}s, iter: {sweep}"
        )
    
    #! make animation
    # if(ensemble_num == 1):
    #     now = time.perf_counter()
    #     get_animation(input, raw_output)
    #     print(
    #         f"animation created, time:"
    #         f" {time.perf_counter()-now}s"
    #     )

    now = time.perf_counter()
    if input.save.save:
        result = get_result(input, processed_input, raw_output, J)
    else:
        result = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, np.array([0.0])

    if(ensemble_num == 1):
        print(
            f"ensemble {ensemble_num} output processed, time:"
            f" {time.perf_counter()-now}s"
        )

    return ensemble_num, result, np.array(autocorr), int(time.perf_counter() - begin)

In [3]:
def sampling(
    input: Input,
    processed_input: Processed_Input,
) -> None:
    
    max_workers, ensemble, irreducible_distance = (
        input.train.max_workers,
        input.train.ensemble,
        processed_input.topology.irreducible_distance
    )

    order, suscept, binder = [], [], []
    spin_order, spin_suscept, spin_binder = [], [], []
    energy, specific = [], []
    corr_time, corr_space = [], []
    time = []
    
    # number, single_result, autocorr, ex_time = run_ensemble(input, processed_input, 1)

    # order.append(single_result[0])
    # suscept.append(single_result[1])
    # binder.append(single_result[2])
    # spin_order.append(single_result[3])
    # spin_suscept.append(single_result[4])
    # spin_binder.append(single_result[5])
    # energy.append(single_result[6])
    # specific.append(single_result[7])
    # corr_space.append(single_result[8])
    # corr_time.append(autocorr)
    # time.append(ex_time)

    with cf.ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(run_ensemble, input, processed_input, i + 1)
            for i in range(ensemble)
        ]

        finished = 0
        for future in cf.as_completed(futures):
            finished += 1
            number, single_result, autocorr, ex_time = future.result()

            print(f"{finished}/{ensemble} finished, ensemble {number}")

            order.append(single_result[0])
            suscept.append(single_result[1])
            binder.append(single_result[2])
            spin_order.append(single_result[3])
            spin_suscept.append(single_result[4])
            spin_binder.append(single_result[5])
            energy.append(single_result[6])
            specific.append(single_result[7])
            corr_space.append(single_result[8])
            corr_time.append(autocorr)
            time.append(ex_time)

    if input.save.save:
        result = Result(
        order_parameter=abs(np.average(order)).item(),
        susceptibility=np.average(suscept).item(),
        binder_cumulant=abs(np.average(binder)).item(),
        spin_glass_order=np.average(spin_order).item(),
        spin_glass_suscept=np.average(spin_suscept).item(),
        spin_glass_binder=np.average(spin_binder).item(),
        energy=np.average(energy).item(),
        specific_heat=np.average(specific).item(),
        irreducible_distance=irreducible_distance,
        correlation_function=column_average_2d(corr_space),
        autocorrelation=column_average_2d(corr_time),
        time=int(np.average(time).item())
    )
        save_log(input, result)        
        save_result(input, result)
        
        print(
        f"T: {input.parameter.T}, Jm: {input.parameter.Jm}, Jv: {input.parameter.Jv}, H: {input.parameter.H}, "
        f"order: {result.order_parameter}, suscept: {result.susceptibility}, binder: {result.binder_cumulant}, "
        f"spin order: {result.spin_glass_order}, spin suscept: {result.spin_glass_suscept}, spin binder: {result.spin_glass_binder}, "
        f"energy: {result.energy}, specific heat: {result.specific_heat}"
        )
    
        print(f"correlation function = {result.correlation_function}")

In [4]:
def experiment(args: argparse.Namespace) -> None:
    lattice = Lattice(
    args.size, args.dimension, 
    args.ghost, args.initial)
    parameter = Parameter(
    args.Tc, args.Hc, args.Jm, args.Jv, args.mode, 
    args.variable, args.multiply, args.base, args.exponent)
    train = Train(
    args.iteration, args.sweep, args.measurement, args.interval,
    args.ensemble, args.max_workers, args.threshold, args.recent)
    save = Save(args.environment, args.location, args.save)
    
    input = Input(lattice, parameter, train, save)
    input.parameter.T, input.parameter.H = args.T, args.H
    
    print("\nexperiment started")
    
    now = time.perf_counter()
    processed_input = get_processed_input(input)
    print(f"input processed, time: {time.perf_counter()-now}s")
    input.parameter.T, input.parameter.H = get_T_and_H(input)

    print(input, "\n")
    sampling(input, processed_input)
    print(f"\nexperiment finished, "
          f"time: {time.perf_counter()-now}s\n")

In [5]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

"""
Memory
Max(measurement * size**dim, size**(2*dim)) * max_workers

Performance
measurement * size**dim * ensembles / max_workers
"""

"""
Lattice Condition
"""
args.size = 32
args.dimension = 2
args.ghost = 0.0
args.initial = "random" # "uniform", "random"

"""
Parameter Condition
"""
args.T, args.H = 1.0, 0.0
# q=2: 2.2692, q=3: 1.4925
# args.Tc = 1/((1-1/args.state)*np.log(1+np.sqrt(args.state)))
args.Tc = 1.1
args.Hc = 0.0
args.Jm = 1.0
args.Jv = 0.0

args.mode = "normal"  # 'normal', 'critical' and 'manual'
args.variable = "T"  # 'T', 'H'
args.multiply = 0.0001
args.base = 2.0
args.exponent = -13

"""
Train Condition
"""
args.iteration = 2**12
args.measurement = 2**14
args.interval = 2**0
args.sweep = args.measurement * args.interval
args.ensemble = 2**9
args.max_workers = 2**3
args.threshold = 5 * 0.1**2
args.recent = 1

"""
Save Condition
"""
args.environment = "local" # "server" or "local"
args.location = "temp" # "result" or "temp"
args.save = True  # True or False

# experiment(args)

# args.multiply, name1, list1 = 0.0001, "exponent", [6, 9, 12]
args.multiply, name1, list1 = 0.0001, "exponent", [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -3, -4, -5, -6, -7, -8, -9, -10, -11]
# args.multiply, name1, list1 = 0.0003, "exponent", [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, -2, -3, -4, -5, -6, -7, -8, -9, -10]
# args.multiply, name1, list1 = 0.0010, "exponent", [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -1, -2, -3, -4, -5, -6, -7, -8]
# args.multiply, name1, list1 = 0.0030, "exponent", [0, 1, 2, 3, 4, 5, 6, 7, 8, -1, -2, -3, -4, -5, -6]
# args.multiply, name1, list1 = 0.0100, "exponent", [0, 1, 2, 3, 4, 5, 6, -1, -2, -3, -4]
# args.multiply, name1, list1 = 0.0300, "exponent", [0, 1, 2, 3, 4, -1, -2, -3]
# args.multiply, name1, list1 = 0.1000, "exponent", [0, 1, 2, 3, -1]

# name1, list1 = "T", [1.1, 1.11, 1.09, 1.12, 1.13, 1.14, 1.15]

for var1 in list1:
    setattr(args, name1, var1)
    experiment(args)


experiment started
input processed, time: 2.552111042001343s
Input(lattice=Lattice(size=32, dimension=2, ghost=0.0, initial='random'), parameter=Parameter(T=1.1008, H=0.0, Tc=1.1, Hc=0.0, Jm=1.0, Jv=0.0, mode='normal', variable='T', multiply=0.0001, base=2.0, exponent=3), train=Train(iteration=4096, sweep=16384, measurement=16384, interval=1, ensemble=512, max_workers=8, threshold=0.05000000000000001, recent=1), save=Save(environment='local', location='temp', save=True)) 

ensemble 1 initial effect removed, time: 0.20280435500171734s, iter: 1, corr: 0.4008, std: 0.0000
ensemble 1 metropolis updated, time: 2.6514424799970584s, iter: 16384
1/512 finished, ensemble 2
2/512 finished, ensemble 3
ensemble 1 output processed, time: 9.424542747001396s
3/512 finished, ensemble 7
4/512 finished, ensemble 1
5/512 finished, ensemble 4
6/512 finished, ensemble 8
7/512 finished, ensemble 5
8/512 finished, ensemble 6
9/512 finished, ensemble 10
10/512 finished, ensemble 9
11/512 finished, ensemble 1

In [ ]:
# import numpy as np
# 
# a = np.random.rand(10, 16)
# # print(a)
# 
# nn_coord = np.array([[1, 3, 4, 12], [2, 0, 5, 13]])
# print(a[:,nn_coord])
# print(a[:,nn_coord][:,:,1]-a[:,nn_coord][:,:,0]+(a[:,nn_coord][:,:,3]-a[:,nn_coord][:,:,2])*1j)
# 
# print(a[:,nn_coord][:,1]-a[:,nn_coord][:,0] + a[:,nn_coord][:,3]-a[:,nn_coord][:,0])

# b = a[np.argmin(
#     np.abs(a[nn_coord][:,1]-a[nn_coord][:,0]),
#     np.abs(2*np.pi-a[nn_coord][:,1]+a[nn_coord][:,0]),
#     np.abs(a[nn_coord][:,1]-a[nn_coord][:,0]-2*np.pi),
#     )] + a[np.argmin((
#     a[nn_coord][:,3]-a[nn_coord][:,2]).abs(),
#     (2*np.pi-a[nn_coord][:,3]+a[nn_coord][:,2]).abs(),
#     (a[nn_coord][:,3]-a[nn_coord][:,2]-2*np.pi).abs(),
#     )]* 1j
# b = (a[:,nn_coord][:,1]-a[:,nn_coord][:,0]) + (a[:,nn_coord][:,3]-a[:,nn_coord][:,2])*1j
# print(b)
# print(b.real, b.imag, np.sqrt(b.real**2+b.imag**2))

In [ ]:
# np.array([[1,2,3,4],[5,6,7,8]]).reshape(2, 2, -1)